In [3]:
import transformers, datasets
print("Transformers:", transformers.__version__)
print("Datasets:", datasets.__version__)
from transformers import TrainingArguments


Transformers: 4.56.1
Datasets: 4.0.0


In [5]:
import sys, inspect, transformers
print("Transformers version/file:", transformers.__version__, transformers.__file__)

from transformers import TrainingArguments
print("TrainingArguments origin:", TrainingArguments.__module__)
print("Interpreter:", sys.executable)
print("Init signature:", inspect.signature(TrainingArguments.__init__))

# Check for local shadows
import glob, os
print("Local transformer-like files:", glob.glob("./transformers*.py*") + glob.glob("./*transformers*/**/*.py", recursive=True)[:5])


Transformers version/file: 4.56.1 C:\Users\abajp\PycharmProjects\Weather_Pipeline\venv\Lib\site-packages\transformers\__init__.py
TrainingArguments origin: transformers.training_args
Interpreter: C:\Users\abajp\PycharmProjects\Weather_Pipeline\venv\Scripts\python.exe
Init signature: (self, output_dir: Optional[str] = None, overwrite_output_dir: bool = False, do_train: bool = False, do_eval: bool = False, do_predict: bool = False, eval_strategy: Union[transformers.trainer_utils.IntervalStrategy, str] = 'no', prediction_loss_only: bool = False, per_device_train_batch_size: int = 8, per_device_eval_batch_size: int = 8, per_gpu_train_batch_size: Optional[int] = None, per_gpu_eval_batch_size: Optional[int] = None, gradient_accumulation_steps: int = 1, eval_accumulation_steps: Optional[int] = None, eval_delay: Optional[float] = 0, torch_empty_cache_steps: Optional[int] = None, learning_rate: float = 5e-05, weight_decay: float = 0.0, adam_beta1: float = 0.9, adam_beta2: float = 0.999, adam_ep

In [ ]:
import numpy as np
from sklearn.metrics import f1_score, classification_report
from transformers import TrainingArguments, Trainer

def make_args():
    common = dict(
        output_dir="models/hf",
        per_device_train_batch_size=16,
        per_device_eval_batch_size=32,
        num_train_epochs=3,
        logging_steps=50,
        save_total_limit=1,
        report_to="none",      # if this errors, try [] or omit
    )
    try:
        # Newer versions accept these kwargs
        return TrainingArguments(**common,
                                 evaluation_strategy="epoch",
                                 save_strategy="epoch",
                                 load_best_model_at_end=True,
                                 metric_for_best_model="f1")
    except TypeError:
        # Fallback for older/odd installs
        return TrainingArguments(**common)

args = make_args()

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return {"f1": f1_score(p.label_ids, preds, average="macro")}

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=ds_tr,
    eval_dataset=ds_va,
    compute_metrics=compute_metrics,
)

trainer.train()

# Explicit eval (needed if we fell back)
eval_metrics = trainer.evaluate()
print("Eval:", eval_metrics)

# Optional detailed report
pred = trainer.predict(ds_va)
y_hat = np.argmax(pred.predictions, axis=1)
print(classification_report(pred.label_ids, y_hat, target_names=labels, digits=4))


Step,Training Loss
50,1.060900
100,0.743300
150,0.596500
200,0.561700
250,0.476900
300,0.552900
350,0.464600
400,0.379700
450,0.428300
500,0.420700
